# A. State preparation with Jaynes-Cummings controls

In [1]:
# ruff: noqa
import os

os.sys.path.append("..")
from feedback_grape.grape_paramaterized import optimize_pulse_parameterized
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

As a preliminary step, we consider state preparation
of a target state starting from a pure state. In addition,
we assume that any coupling to an external environment
is negligible and that the parametrized controls can be
implemented perfectly.

Here no feedback is required, we are just testing the parameterized gates setup.

As a first example, we consider the state preparation
of a cavity resonantly coupled to an externally driven
qubit

Here, we consider a particular sequence of
parametrized unitary gates originally introduced by Law
and Eberly

In [2]:
N_cav = 4

In [3]:
def qubit_unitary(alpha):
    """
    TODO: see if alpha, can be sth elser other than scalar, and if the algo understands this
    see if there can be multiple params like alpha and beta input
    """
    return expm(
        -1j
        * (
            alpha * tensor(identity(N_cav), sigmap())
            + alpha.conjugate() * tensor(identity(N_cav), sigmam())
        )
        / 2
    )

In [4]:
def qubit_cavity_unitary(beta):
    return expm(
        -1j
        * (
            beta
            * (
                tensor(destroy(N_cav), identity(2))
                @ tensor(identity(N_cav), sigmap())
            )
            + beta.conjugate()
            * (
                tensor(create(N_cav), identity(2))
                @ tensor(identity(N_cav), sigmam())
            )
        )
        / 2
    )

In [5]:
alpha = 0.1 + 0.1j
beta = 0.1 + 0.1j
Uq = qubit_unitary(alpha)
Uqc = qubit_cavity_unitary(beta)
print(
    "Uq unitary check:",
    jnp.allclose(Uq.conj().T @ Uq, jnp.eye(Uq.shape[0]), atol=1e-7),
)
print(
    "Uqc unitary check:",
    jnp.allclose(Uqc.conj().T @ Uqc, jnp.eye(Uqc.shape[0]), atol=1e-7),
)

Uq unitary check: True
Uqc unitary check: True


In [6]:
qubit_unitary(0.1).shape

(8, 8)

In [7]:
qubit_unitary(0.1)

Array([[0.99875026+0.j        , 0.        -0.04997917j,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        -0.04997917j, 0.99875026+0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.99875026+0.j        , 0.        -0.04997917j,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        -0.04997917j, 0.99875026+0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0

In their groundbreaking work, Law and Eberly have
shown that any arbitrary superposition of Fock states with
maximal excitation number N can be prepared out of the
ground state in a sequence of N such interleaved gates,
also providing an algorithm to find the correct angles and
interaction durations

In [8]:
print(qubit_cavity_unitary(0.1).shape)

(8, 8)


In [9]:
qubit_cavity_unitary(0.1)

Array([[0.99875026+0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        -0.04997917j,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 1.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.99750104+0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        -0.07065177j,
        0.        +0.j        , 0.        +0.j        ],
       [0.        -0.04997917j, 0.        +0.j        ,
        0.        +0.j        , 0.99875026+0.j        ,
        0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0

## First target is the state $ | 1 ⟩ + | 3 ⟩ $ 

In [10]:
time_steps = 3  # corressponds to maximal excitation number of an arbitrary Fock State Superposition

In [11]:
psi0 = tensor(basis(N_cav), basis(2))
psi_target = tensor((fock(N_cav, 1) + fock(N_cav, 3)), basis(2))

In [12]:
psi0.shape

(8, 1)

In [13]:
psi_target.shape

(8, 1)

In [14]:
print(fock(N_cav, 1))

[[0.+0.j]
 [1.+0.j]
 [0.+0.j]
 [0.+0.j]]


Law and Eberly provided an algorithm to determine the correct parameters for state preparation. These include:

- The rotation angle $ |\alpha| $,
- The azimuthal angle $ \arg\left(\frac{\alpha}{|\alpha|}\right) $,
- The interaction duration $ |\beta| $. <br>

So Goal is to find the best control vector (rather than control amplitudes, this time) that leads to finding the optimal state-preparation strategies. Performing as well as the Law-Eberly algorithm.

In [15]:
# TODO: see why if initial [paramaters are complex, adam fidelity is so bad and l-bfgs does error]
import jax.random as random

key = random.PRNGKey(0)  # Random seed for reproducibility
num_gates = len(
    [qubit_unitary, qubit_cavity_unitary]
)  # Number of parameterized gates
initial_parameters = jnp.full((time_steps, num_gates), 0.1, dtype=jnp.float64)
result_shapes = [
    gate(initial_parameters[step, i]).shape
    for step in range(time_steps)
    for i, gate in enumerate([qubit_unitary, qubit_cavity_unitary])
]
print(f"Shapes of results: {result_shapes}")

Shapes of results: [(8, 8), (8, 8), (8, 8), (8, 8), (8, 8), (8, 8)]


In [16]:
print(initial_parameters)

[[0.1 0.1]
 [0.1 0.1]
 [0.1 0.1]]


## Optimizing
Currently l-bfgs with the same learning rate of 0.3 converges at a local minimum of 0.5, adam also converges at 0.5 but at smaller learning rates

In [17]:
result = optimize_pulse_parameterized(
    U_0=psi0,
    C_target=psi_target,
    parameterized_gates=[qubit_unitary, qubit_cavity_unitary],
    initial_parameters=initial_parameters,
    num_time_steps=time_steps,
    optimizer="adam",
    max_iter=1000,
    convergence_threshold=1e-6,
    type="state",
    propcomp="time-efficient",
    learning_rate=0.3,
)

In [18]:
print(result)

result(control_amplitudes=Array([[-3.15190385,  4.32204244],
       [ 3.15093185,  1.44659991],
       [ 3.13826641, -1.81466037]], dtype=float64), final_fidelity=Array(0.99997576, dtype=float64), iterations=189, final_operator=Array([[-2.39002192e-03+0.j        ],
       [ 0.00000000e+00+0.00179682j],
       [-7.05011908e-01+0.j        ],
       [ 0.00000000e+00+0.00220421j],
       [ 2.60457850e-04+0.j        ],
       [ 0.00000000e+00-0.00052837j],
       [-7.09184512e-01+0.j        ],
       [ 0.00000000e+00-0.00117946j]], dtype=complex128))


In [19]:
from feedback_grape.grape_paramaterized import calculate_trajectory
from feedback_grape.utils.fidelity import fidelity

U_final = calculate_trajectory(
    psi0,
    result.control_amplitudes,
    time_steps,
    [qubit_unitary, qubit_cavity_unitary],
    type="state",
)
print(fidelity(C_target=psi_target, U_final=U_final, type="state"))

0.9999757572313498
